In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import PROJECT.util as f
from matplotlib.pyplot import cm
from mpl_toolkits import mplot3d
import plotly.graph_objects as go
import pandas as pd
import pickle
from plotly.subplots import make_subplots
import bz2

In [ ]:
def overlap(x,x_,d):
    return np.abs(np.einsum('i,ji->j', x, x_)/d)

def mse1(x_e, array_x):
    ## x-y
    mse=((np.abs(x_e)-np.abs(array_x))**2).mean(axis=1)
    return mse
def mse2(u_e, v_e, array_u, array_v, Y):
    ## u*v*-uv
    mse = np.tensordot(u_e, v_e, 0) - np.einsum('ad,ah->adh',array_u, array_v)
    return (mse**2).mean(axis=(1,2))
def mse3(u_t, v_t, Y):
    ## Y-uv
    tmp = Y - np.einsum('ad,ah->adh',array_u, array_v)
    return (mse**2).mean(axis=(1,2))

In [ ]:
# WHERE TO READ
FILE_PATH="C:/Users/Admin/Desktop/"
# WHERE TO WRITE THE PLOTS
FILE_PATH_W="C:/Users/Admin/Desktop/graph/"

# enter the configuration
name_n_1="200"
name_m_1="200"
lambda_u_val_1=1
lambda_v_val_1=1
lambda_val_1="2"

name_value_1="beta_u"
name_value_2="beta_v"

filename =f"n={name_n_1}_m={name_m_1}_lu={lambda_u_val_1}_lv={lambda_v_val_1}_l={lambda_val_1}.pickle"

infile = bz2.BZ2File(FILE_PATH+filename,'rb')
(main_res_u_new_1, main_res_v_new_1, main_config_new_1, save_checking_range_new_1) = pickle.load(infile)
infile.close()

In [ ]:
# enter the second configuration
name_n_2="200"
name_m_2="200"
lambda_u_val_2=1
lambda_v_val_2=1
lambda_val_2="2"

# Pick a mse
name_mse='mse1'

filename =f"n={name_n_2}_m={name_m_2}_lu={lambda_u_val_2}_lv={lambda_v_val_2}_l={lambda_val_2}.pickle"

infile = bz2.BZ2File(FILE_PATH+filename,'rb')
(main_res_u_new_2, main_res_v_new_2, main_config_new_2, save_checking_range_new_2) = pickle.load(infile)
infile.close()

In [ ]:
list_value_1=save_checking_range_new_1[0]
list_value_2=save_checking_range_new_1[1]

list_value_1_2=save_checking_range_new_2[0]
list_value_2_2=save_checking_range_new_2[1]

if(all(list_value_1!=list_value_1_2) or all(list_value_2!=list_value_2_2)):
    raise NameError('The ranges should be the same!')

In [ ]:
## COMPUTE THE CHOOSEN MSE:
mse_at_each_iteration_1=[]
mse_at_each_iteration_u_1=[]
mse_at_each_iteration_v_1=[]

mse_at_each_iteration_2=[]
mse_at_each_iteration_u_2=[]
mse_at_each_iteration_v_2=[]
if(name_mse=='mse1'):
    mse_at_each_iteration_u_1=[]
    mse_at_each_iteration_v_1=[]
    
    mse_at_each_iteration_u_2=[]
    mse_at_each_iteration_v_2=[]
    for i in range(len(main_config_new_1)):
        # intermediate array to split the main array:
        # shape(nb_run*sample_size) to shape((nb_run, sample_size))
        one_sample_u=[]
        one_sample_v=[]
        
        one_sample_u_2=[]
        one_sample_v_2=[]
        for j in range(len(main_config_new_1[0])):
            one_sample_u.append(mse1(main_config_new_1[i][j][0]['u_'],
                                      main_res_u_new_1[i][j]))
            one_sample_v.append(mse1(main_config_new_1[i][j][0]['v_'],
                                      main_res_v_new_1[i][j]))
            
            one_sample_u_2.append(mse1(main_config_new_2[i][j][0]['u_'],
                                      main_res_u_new_2[i][j]))
            one_sample_v_2.append(mse1(main_config_new_2[i][j][0]['v_'],
                                      main_res_v_new_2[i][j]))
            
        mse_at_each_iteration_u_1.append(one_sample_u)
        mse_at_each_iteration_v_1.append(one_sample_v)
        
        mse_at_each_iteration_u_2.append(one_sample_u_2)
        mse_at_each_iteration_v_2.append(one_sample_v_2)
elif(name_mse=='mse2'):
    mse_at_each_iteration_1=[]
    
    mse_at_each_iteration_2=[]
    for i in range(len(main_config_new_1)):
        # intermediate array to split the main array:
        # shape(nb_run*sample_size) to shape((nb_run, sample_size))
        one_sample=[]
        
        one_sample_2=[]
        for j in range(len(main_config_new_1[0])):
            one_sample.append(mse2(main_config_new_1[i][j][0]['u_'],
                                      main_config_new_1[i][j][0]['v_'],
                                      main_res_u_new_1[i][j],
                                      main_res_v_new_1[i][j],
                                      main_config_new_1[i][j][0]['Y']))
            
            one_sample_2.append(mse2(main_config_new_2[i][j][0]['u_'],
                                      main_config_new_2[i][j][0]['v_'],
                                      main_res_u_new_2[i][j],
                                      main_res_v_new_2[i][j],
                                      main_config_new_2[i][j][0]['Y']))
            
        mse_at_each_iteration_1.append(one_sample)
        mse_at_each_iteration_2.append(one_sample_2)
elif(name_mse=='mse3'):
    mse_at_each_iteration_1=[]
    mse_at_each_iteration_2=[]
    for i in range(len(main_config_new_1)):
        # intermediate array to split the main array:
        # shape(nb_run*sample_size) to shape((nb_run, sample_size))
        one_sample=[]
        one_sample_2=[]
        for j in range(len(main_config_new_1[0])):
            one_sample.append(mse3(main_res_u_new_1[i][j],
                                   main_res_v_new_1[i][j],
                                   main_config_new_1[i][j][0]['Y']))
            one_sample_2.append(mse3(main_res_u_new_2[i][j],
                                   main_res_v_new_2[i][j],
                                   main_config_new_2[i][j][0]['Y']))
        mse_at_each_iteration_1.append(one_sample)
        mse_at_each_iteration_2.append(one_sample_2)

In [ ]:
mse_at_each_iteration_1=np.array(mse_at_each_iteration_1)
mse_at_each_iteration_u_1=np.array(mse_at_each_iteration_u_1)
mse_at_each_iteration_v_1=np.array(mse_at_each_iteration_v_1)

mse_at_each_iteration_2=np.array(mse_at_each_iteration_2)
mse_at_each_iteration_u_2=np.array(mse_at_each_iteration_u_2)
mse_at_each_iteration_v_2=np.array(mse_at_each_iteration_v_2)

if(name_mse=='mse1'):
    # MSE U
    mse_at_final_iteration_u_1 = mse_at_each_iteration_u_1[:,:,-1].mean(axis=1)
    mse_at_final_iteration_u_2 = mse_at_each_iteration_u_2[:,:,-1].mean(axis=1)

    # MSE V
    mse_at_final_iteration_v_1 = mse_at_each_iteration_v_1[:,:,-1].mean(axis=1)
    mse_at_final_iteration_v_2 = mse_at_each_iteration_v_2[:,:,-1].mean(axis=1)
    
    # BOTH
    mse_at_each_iteration_1 = np.mean([mse_at_each_iteration_u_1, mse_at_each_iteration_v_1], axis=0)
    mse_at_each_iteration_2 = np.mean([mse_at_each_iteration_u_2, mse_at_each_iteration_v_2], axis=0)
elif(name_mse=='mse2' or name_mse=='mse3'):
    # MSE U and V are the same in this configuration:
    mse_at_each_iteration_u_1 = mse_at_each_iteration_1
    mse_at_final_iteration_u_1 = mse_at_each_iteration_u_1[:,:,-1].mean(axis=1)
    mse_at_each_iteration_v_1 = mse_at_each_iteration_1
    mse_at_final_iteration_v_1 = mse_at_each_iteration_v_1[:,:,-1].mean(axis=1)
    
    mse_at_each_iteration_u_2 = mse_at_each_iteration_2
    mse_at_final_iteration_u_2 = mse_at_each_iteration_u_2[:,:,-1].mean(axis=1)
    mse_at_each_iteration_v_2 = mse_at_each_iteration_2
    mse_at_final_iteration_v_2 = mse_at_each_iteration_v_2[:,:,-1].mean(axis=1)

# Take last iteration and mean between sample:
mse_at_final_iteration_1 = mse_at_each_iteration_1[:,:,-1].mean(axis=1)
mse_at_final_iteration_2 = mse_at_each_iteration_2[:,:,-1].mean(axis=1)

In [ ]:
## COMPUTE OVERLAP AT EACH ITERATION

# OVERLAP FOR U:
overlap_at_each_iteration_u_1=[]
overlap_at_each_iteration_u_2=[]
for i in range(len(main_config_new_1)):
    # intermediate array to split the main array:
    # shape(nb_run*sample_size) to shape((nb_run, sample_size))
    one_sample=[]
    one_sample_2=[]
    for j in range(len(main_config_new_1[0])):
        one_sample.append(overlap(main_config_new_1[i][j][0]['u_'],
                                  main_res_u_new_1[i][j],
                                  main_config_new_1[i][j][0]['N']))
        one_sample_2.append(overlap(main_config_new_2[i][j][0]['u_'],
                                  main_res_u_new_2[i][j],
                                  main_config_new_2[i][j][0]['N']))
    overlap_at_each_iteration_u_1.append(one_sample)
    overlap_at_each_iteration_u_2.append(one_sample_2)
    
# OVERLAP FOR U:
overlap_at_each_iteration_v_1=[]
overlap_at_each_iteration_v_2=[]
for i in range(len(main_config_new_1)):
    # intermediate array to split the main array:
    # shape(nb_run*sample_size) to shape((nb_run, sample_size))
    one_sample=[]
    one_sample_2=[]
    for j in range(len(main_config_new_1[0])):
        one_sample.append(overlap(main_config_new_1[i][j][0]['v_'],
                                  main_res_v_new_1[i][j],
                                  main_config_new_1[i][j][0]['N']))
        one_sample_2.append(overlap(main_config_new_2[i][j][0]['v_'],
                                  main_res_v_new_2[i][j],
                                  main_config_new_2[i][j][0]['N']))
    overlap_at_each_iteration_v_1.append(one_sample)
    overlap_at_each_iteration_v_2.append(one_sample_2)

In [ ]:
# OVERLAP U
overlap_at_each_iteration_u_1 = np.array(overlap_at_each_iteration_u_1)
overlap_at_final_iteration_u_1 = overlap_at_each_iteration_u_1[:,:,-1].mean(axis=1)
overlap_at_each_iteration_u_2 = np.array(overlap_at_each_iteration_u_2)
overlap_at_final_iteration_u_2 = overlap_at_each_iteration_u_2[:,:,-1].mean(axis=1)

# OVERLAP V
overlap_at_each_iteration_v_1 = np.array(overlap_at_each_iteration_v_1)
overlap_at_final_iteration_v_1 = overlap_at_each_iteration_v_1[:,:,-1].mean(axis=1)
overlap_at_each_iteration_v_2 = np.array(overlap_at_each_iteration_v_2)
overlap_at_final_iteration_v_2 = overlap_at_each_iteration_v_2[:,:,-1].mean(axis=1)

# BOTH
overlap_at_each_iteration_1 = np.mean([overlap_at_each_iteration_u_1, overlap_at_each_iteration_v_1], axis=0)
overlap_at_each_iteration_2 = np.mean([overlap_at_each_iteration_u_2, overlap_at_each_iteration_v_2], axis=0)

# Take last iteration and mean between sample:
overlap_at_final_iteration_1 = overlap_at_each_iteration_1[:,:,-1].mean(axis=1)
overlap_at_final_iteration_2 = overlap_at_each_iteration_2[:,:,-1].mean(axis=1)

In [ ]:
######################### 3D PLOT BOTH CONFIG

x, y = np.meshgrid(list_value_2,list_value_1)
overlap_at_final_iteration_1=overlap_at_final_iteration_1.reshape(len(list_value_1),len(list_value_2))

overlap_at_final_iteration_2=overlap_at_final_iteration_2.reshape(len(list_value_1),len(list_value_2))

fig = go.Figure(data=[
    go.Surface(name=f"n={name_n_1}, m={name_m_1}; lu={lambda_u_val_1}, lv={lambda_v_val_1}, l={lambda_val_1}",
               z=overlap_at_final_iteration_1,x=list_value_1, y=list_value_2,
               colorscale ='Blues', showscale=False, opacity=1),
    go.Surface(name=f"n={name_n_2}, m={name_m_2}; lu={lambda_u_val_2}, lv={lambda_v_val_2}, l={lambda_val_2}",
               z=overlap_at_final_iteration_2,x=list_value_1, y=list_value_2,
               colorscale ='Greens', showscale=False, opacity=0.9)])

fig.update_layout(
    title=f"OVERLAP:",
)

fig.update_layout(showlegend=True, scene = dict(
                    xaxis_title=name_value_2,
                    yaxis_title=name_value_1,
                    zaxis_title="overlap"))
fig.update_traces(showlegend=True)

fig.show()
path=f"{FILE_PATH_W}R1_n={name_n_1}_m={name_m_1}_"\
f"Ldiff={lambda_u_val_1/lambda_v_val_1}_"\
f"R2_n={name_n_2}_m={name_m_2}_"\
f"Ldiff={lambda_u_val_2/lambda_v_val_2}_overlap.html"
fig.write_html(path)

In [ ]:
######################### 3D PLOT BOTH CONFIG
#### MSE
x, y = np.meshgrid(list_value_2,list_value_1)
mse_at_final_iteration_1=mse_at_final_iteration_1.reshape(len(list_value_1),len(list_value_2))

mse_at_final_iteration_2=mse_at_final_iteration_2.reshape(len(list_value_1),len(list_value_2))

fig = go.Figure(data=[
    go.Surface(name=f"n={name_n_1}, m={name_m_1}; lu={lambda_u_val_1}, lv={lambda_v_val_1}, l={lambda_val_1}",
               z=mse_at_final_iteration_1,x=list_value_1, y=list_value_2,
               colorscale ='Blues', showscale=False, opacity=1),
    go.Surface(name=f"n={name_n_2}, m={name_m_2}; lu={lambda_u_val_2}, lv={lambda_v_val_2}, l={lambda_val_1}",
               z=mse_at_final_iteration_2,x=list_value_1, y=list_value_2,
               colorscale ='Greens', showscale=False, opacity=0.9)])

fig.update_layout(
    title=f"{name_mse}:",
)
fig.update_layout(scene = dict(
                    xaxis_title=name_value_2,
                    yaxis_title=name_value_1,
                    zaxis_title="mse"))
fig.update_traces(showlegend=True)

fig.show()
path=f"{FILE_PATH_W}R1_n={name_n_1}_m={name_m_1}_"\
f"Ldiff={lambda_u_val_1/lambda_v_val_1}_"\
f"R2_n={name_n_2}_m={name_m_2}_"\
f"Ldiff={lambda_u_val_2/lambda_v_val_2}_mse.html"
#fig.write_html(path)